In [1]:
import numpy as np
from scipy.stats import invgamma
from sklearn.preprocessing import normalize

In [2]:
filePath = "data/ml-100k/u1.base"
with open(filePath, "rt") as dataPath:
    raw_data = dataPath.read().splitlines()
datapoints = [[int(i) for i in data.split("\t")] for data in raw_data]

# indexing on users/movies starts at 1, reset to index from 0, this will be important when we do testing
datapoints = [[row[0] - 1, row[1] - 1, row[2], row[3]] for row in datapoints]

user_ids = set([datapoint[0] for datapoint in datapoints])
items_ids = set([datapoint[1] for datapoint in datapoints])

In [3]:
user_ratings = {user_id: {} for user_id in user_ids}
for user_id, item_id, rating, timestamp in datapoints:
    user_ratings[user_id][item_id] = (rating, timestamp)

In [4]:
item_ratings = {item_id: {} for item_id in items_ids}
for user_id, item_id, rating, timestamp in datapoints:
    item_ratings[item_id][user_id] = (rating, timestamp)

In [5]:
def precision(var, v_j, var_u, k):
    first = 1 / var * np.dot(v_j, v_j.T)
    second = 1 / var_u * np.eye(k)
    if v_j.shape[1] > 0:
        return first * second
    return second

def eta(r_ij, v_j, k):
    return np.sum(np.array(v_j) * np.array(r_ij), axis=1)
    
def mu(var, prec, et):
    if np.array_equal(prec, np.zeros(prec.shape)):
        return np.zeros(prec.shape[0])
    return np.dot(1 / var * np.linalg.inv(prec), et)
    
def SampleUi(r_ij, v_j, var_u, var, k):
    prec = precision(var, v_j, var_u, k)
    et = eta(r_ij, v_j, k)
    m = mu(var, prec, et)
    return np.random.multivariate_normal(m, prec)

In [6]:
def softmax(x):
    # subtract max value to prevent overflow
    return np.exp(x - np.max(x)) / np.sum(np.exp(x - np.max(x)))
#     s = np.sum([np.exp(i) for i in numpy_arr])
#     return np.array([np.exp(i) / s for i in numpy_arr]).reshape(-1)

In [7]:
def stratified_resample(weights):
    N = len(weights)
    # make N subdivisions, and chose a random position within each one
    positions = (np.random.random(N) + range(N)) / N

    indexes = np.zeros(N, 'i')
    cumulative_sum = np.cumsum(weights)
    i, j = 0, 0
    while i < N:
        if positions[i] < cumulative_sum[j]:
            indexes[i] = j
            i += 1
        else:
            j += 1
    return indexes


def systematic_resample(weights):
    N = len(weights)

    # make N subdivisions, and choose positions with a consistent random offset
    positions = (np.random.random_sample() + np.arange(N)) / N

    indexes = np.zeros(N, 'i')
    cumulative_sum = np.cumsum(weights)
    i, j = 0, 0
    while i < N:
        if positions[i] < cumulative_sum[j]:
            indexes[i] = j
            i += 1
        else:
            j += 1
    return indexes

In [8]:
def sample_var(matrix, alpha, beta, k, n_users):
    alpha = n_users * k / 2.0 + alpha
    beta = 0.5 * np.linalg.norm(matrix, 'fro') + beta
    # Generate random value from inverse gamma(shape,scale)
    var = invgamma.rvs(alpha, scale=beta)
    return var

In [12]:
var = 1.0
var_u = 1.0
var_v = 1.0
k = 3
n_particles = 5
alpha = 2
beta = 0.5

tot_reward = 0

particles = [{"u": np.random.normal(size=(len(user_ids), k)),
              "v": np.random.normal(size=(k, len(items_ids))),
              "var_u": 1.0,
              "var_v": 1.0,
              "w": 1 / n_particles} for particle in range(n_particles)]

# user_ratings_seen = {}
users = {}
items = {}

for _idx, (user_id, item_id, rating, timestamp) in enumerate(datapoints):
    
    if user_id not in users:
        users[user_id] = {"movies_seen": [], "rewards": []}
    user_data = users[user_id]
        
    particle_ind = np.random.choice(range(n_particles), p=[x["w"] for x in particles])
    
    v_tilde = particles[particle_ind]["v"]
    
    v_j = v_tilde[:, user_data["movies_seen"]]
    
    r_ij = user_data["rewards"]
    
    u_i = SampleUi(r_ij, v_j, particles[particle_ind]["var_u"], var, k)
    
    pred = np.dot(u_i, v_j)
    
    try:
        j = np.argmax(pred)
    except:
        # if argmax fails, it's because u_i and v_j hav no data yet, just suggest something at random
        j = np.random.randint(len(items_ids))
    
    
    if j in user_ratings[user_id] and user_ratings[user_id][j][0] > 2.5:
        reward = 1
    else:
        reward = 0
        
    tot_reward += reward
    
    if _idx % 1001 == 0:
        print("current regret: ", _idx - tot_reward)
        
    users[user_id]["rewards"].append(reward)
    users[user_id]["movies_seen"].append(item_id)
    
    try:
        items[j]["rewards"].append(reward)
        items[j]["users_seen"].append(user_id)
    except:
        items[j] = {"rewards": [reward], "users_seen": [user_id]}
    
    # update posterior
    lambdas = []
    for particle in particles:
        one_over_sig_sqr = np.eye(k) * 1 / particle["var_u"]
        if len(user_data["movies_seen"]) == 0:
            lambdas.append(one_over_sig_sqr)
        else:
            v_js = particle["v"][:, user_data["movies_seen"]]
            lambdas.append(np.dot(v_js, v_js.T) * 1 / var)
        
    etas = []
    movies_seen = user_data["movies_seen"]
    for particle in particles:
        # first movie for this user, eta should be zero
        if len(movies_seen) == 0:
            etas.append(np.zeros(k))
        else:
            e = eta(r_ij, particle["v"][:, movies_seen], k)
            etas.append(e)
    
    weights = []
    for idx, particle in enumerate(particles):
        try:
            loc = np.dot(particle["v"][:, j], mu(var, lambdas[idx], etas[idx]))
        except:
            loc = np.zeros(1)
        scale = 1 / var * np.dot(np.dot(particle["v"][:, j].T, lambdas[idx]), particle["v"][:, j])
        weights.append(np.random.normal(loc, scale))
    
    weights = softmax(normalize(np.array(weights).reshape(-1, 1)))
    
    particle_indices = systematic_resample(weights)
    
    new_particles = []
    for ind in particle_indices:
        particle = particles[ind]
        new_particles.append({
            "u": particle["u"],
            "v": particle["v"],
            "var_u": particle["var_u"],
            "var_v": particle["var_v"],
            "w": 1 / n_particles,
            "lambda": lambdas[ind],
            "eta": etas[ind]
        })
    particles = new_particles
    
    for particle in particles:
        lambda_upd = 1 / var * np.dot(v_tilde[:, j].reshape(-1, 1), v_tilde[:, j].reshape(1, -1))
        particle["lambda"] += lambda_upd
        eta_update = reward * v_tilde[:, j]
        particle["eta"] += eta_update
        u_i = SampleUi(users[user_id]["rewards"],
                       particle["v"][:, user_data["movies_seen"]],
                       particle["var_u"],
                       var,
                       k)
        particle["u"][user_id, :] = u_i
        u_ij = particle["u"][items[j]["users_seen"], :]
        prec_v = np.dot(u_ij.T, u_ij) + 1 / particle["var_v"] * np.eye(k)
        et = np.dot(u_ij.T, np.array(items[j]["rewards"]))
        v_j = np.random.multivariate_normal(var * np.dot(np.linalg.inv(prec_v), et), np.linalg.inv(prec_v))
        particle["v"][:, j] = v_j
        particle["var_u"] = sample_var(particle["u"], alpha, beta, k, len(user_ids))

current regret:  0
current regret:  808
current regret:  1637
current regret:  2527
current regret:  3369
current regret:  4252
current regret:  5023
current regret:  5885
current regret:  6762
current regret:  7700
current regret:  8632
current regret:  9489
current regret:  10345
current regret:  11257
current regret:  12117
current regret:  12964
current regret:  13760
current regret:  14633
current regret:  15377
current regret:  16273
current regret:  17165
current regret:  18019
current regret:  18860
current regret:  19710
current regret:  20600
current regret:  21421
current regret:  22210
current regret:  22992
current regret:  23741
current regret:  24526
current regret:  25283
current regret:  25935
current regret:  26665
current regret:  27493
current regret:  28244
current regret:  29021
current regret:  29859
current regret:  30642
current regret:  31453
current regret:  32271
current regret:  33094
current regret:  33771
current regret:  34584
current regret:  35391
curr

IndexError: index 1650 is out of bounds for axis 1 with size 1650